In [1]:
import json
import codecs
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import silhouette_score
#pd.options.display.max_rows = 50
#pd.reset_option('display')

<h2>Gathering the Data</h2>
<p>In this initial section, we will gather all of the cuneiform sign transliterations from the JSON files in our dataset. Then we will consolidate them into a data frame and add other columns to help our computations.</p>

<p>First, we will load a map from sign value to sign name to use on the signs in our texts. The OGSL is... (website...)</p>

In [2]:
file_ogsl = codecs.open('ogsl-sl.json','r','utf-8')
ogsl = json.load(file_ogsl)
sign_index = ogsl['index']

<h3>Collect the Text Signs</h3>
<p>The following code parses the JSON files of the ORACC texts and collects each sign transliteration. Since different signs have different types of reading, they are rendered differently in the JSON file and we must take care to recognize each sign reading type in the JSON file</p>
The types of signs and their representation in the JSON Files:
<ol>
    <li>Syllable - The reading of a sign as a syllable is rendered with a 'v' key</li>
    <li>Logogram - The reading of a sign as a logogram, i.e. one represents a word in itself or as part of a complex of signs that represents a single word is written in capital letters and with a 's' key</li>
    <li>Numerical - A sign representing a number (or personal name determinative) has an extra key called 'sexified'. This gives information on the number sign's wedge structure.</li>
</ol>

In addition, a modified sign can be any of the three types above, but written with a nonstandard paleography (e.g. a diagonal wedge is incised in the clay instead of a horizontal). These are the signs we want to examine. They have extra data given under the 'mods' key.

In [3]:
def process_signs(sign_data):
    sign_info = {}
    if 'v' in sign_data: #This is the label for a standard syllable
        sign_info['b'] = sign_data['v']
    if 's' in sign_data: #This is the label for elements of a logogram
        sign_info['b'] = sign_data['s']
    if 'n' in sign_data:
        sign_info['b'] = sign_data.get('sexified',sign_data.get('form','noform?'))
    if 'mods' in sign_data:
        for m in sign_data['mods']:
            for d in m:
                sign_info[d] = m[d]
    if 'break' in sign_data:
        sign_info['break'] = sign_data['break']
    sign_info['sign_loc_id'] = sign_data.get('id','no-id')
    return sign_info    

In [4]:
types = set()
all_signs = []
all_words = []
for fname in os.listdir('sargonletters/corpusjson'):
    f = codecs.open('sargonletters/corpusjson/'+fname,'r','utf-8')
    try:
        j = json.load(f)
    except ValueError:
        print('Could not load: ' + fname)
        continue
    text_id = j['textid']
    for a in j['cdl'][0]['cdl']:
        if a.get('type','') == 'discourse':
            for b in a['cdl']:
                if b.get('type','') == 'sentence':
                    line_label = ''                    
                    for c in b['cdl']:
                        if c.get('node','') == 'd': #This is the label for the line e.g. "o ii 3"
                            line_label = c.get('label','nolabel')
                        if c.get('node','') == 'l': #This is the label for a regular word in a line
                            if c.get('tail-sig','') != '': #An extra word??
                                continue
                            form = c['f']['form']
                            frag = c['frag']
                            ref  = c['ref']
                            cf   = c['f'].get('cf','no-cf')
                            gw   = c['f'].get('gw','no-gw')
                            pos  = c['f']['pos']
                            sense = c['f'].get('sense','no-sense')
                            norm = c['f'].get('norm','no-norm')
                            epos = c['f'].get('epos','no-epos')
                            word_sign_tot = len(c['f']['gdl'])
                            word_info = {'file':fname,'line_label':line_label,'form': form,'frag': frag, 'text_id': text_id, 'ref': ref,'cf': cf,'gw': gw,'pos': pos,'epos':epos,'sense':sense,'word_sign_tot':word_sign_tot,'norm':norm}
                            all_words.append(word_info)
                            for sign_data in c['f']['gdl']:
                                if sign_data.get('det','') == 'semantic':
                                    for sd in sign_data['seq']:
                                        if sd.get('gg','') == 'logo':
                                            for g in sd['group']:
                                                sign_info = process_signs(g)
                                                sign_info.update(word_info)
                                                all_signs.append(sign_info)
                                        else:
                                            sign_info = process_signs(sd)
                                            sign_info.update(word_info)
                                            all_signs.append(sign_info)
                                elif sign_data.get('gg','') == 'logo':
                                    for g in sign_data['group']:
                                        if g.get('det','') == 'semantic':
                                            for sd in g['seq']:
                                                if sd.get('gg','') == 'logo':
                                                    for gg in sd['group']:
                                                        sign_info = process_signs(gg)
                                                        sign_info.update(word_info)
                                                        all_signs.append(sign_info)                                       
                                                else:
                                                    sign_info = process_signs(sd)
                                                    sign_info.update(word_info)
                                                    all_signs.append(sign_info)                                       
                                        else:
                                            sign_info = process_signs(g)
                                            sign_info.update(word_info)
                                            all_signs.append(sign_info)                                        
                                else:
                                    sign_info = process_signs(sign_data)
                                    sign_info.update(word_info)
                                    all_signs.append(sign_info)
                        '''
                        if c.get('node','') == 'c': #This is the label for a phrase. This seems to no longer be used 
                            for d in c['cdl']:
                                if d.get('node','') == 'l':
                                    form = d['f']['form']
                                    for sign_data in d['f']['gdl']:
                                        if sign_data.get('det','') == 'semantic':
                                            for sd in sign_data['seq']:
                                                sign_info = process_signs(sd)
                                                sign_info.update({'file':fname,'line_label':line_label,'form': form})
                                                all_signs.append(sign_info)
                                        else:
                                            sign_info = process_signs(sign_data)
                                            sign_info.update({'file':fname,'line_label':line_label,'form': form})
                                            all_signs.append(sign_info)
                        '''
                        #types.add(c.get('type','no type'))
                        
print('done')

Could not load: P314095.json
done


Now, we form our Data Frame where each row contains information on every sign in the corpus. Further limitations on which signs are significant to our purposes will be made later, but for now we will eliminate all of the signs which are labelled as "missing," (i.e. reconstructed) because any information based on their paleography or orthography cannot be ascertained.

In [5]:
df = pd.DataFrame(all_signs)
df = df.fillna('')
df

,a,b,break,cf,epos,f,file,form,frag,gw,line_label,m,norm,pos,ref,sense,sign_loc_id,text_id,word_sign_tot
0,,a,damaged,awātu,N,,P224485.json,a-bat,⸢a⸣-bat,word,o 1,,abat,N,P224485.2.1,word,P224485.2.1.0,P224485,2
1,,bat,,awātu,N,,P224485.json,a-bat,⸢a⸣-bat,word,o 1,,abat,N,P224485.2.1,word,P224485.2.1.1,P224485,2
2,,LUGAL,,šarru,N,,P224485.json,LUGAL,LUGAL,king,o 1,,šarri,N,P224485.2.2,king,P224485.2.2.0,P224485,1
3,,a,,ana,PRP,,P224485.json,a-na,a-na\t,to,o 1,,ana,PRP,P224485.2.3,to,P224485.2.3.0,P224485,2
4,,na,,ana,PRP,t,P224485.json,a-na,a-na\t,to,o 1,,ana,PRP,P224485.2.3,to,P224485.2.3.1,P224485,2
5,,1(diš),,Aššur-šarru-uṣur,PN,,P224485.json,{1}aš-šur-MAN-PAB,{1}aš-šur-MAN-⸢PAB,1,o 1,,Aššur-šarru-uṣur,PN,P224485.2.4,1,P224485.2.4.0,P224485,4
6,,aš,,Aššur-šarru-uṣur,PN,,P224485.json,{1}aš-šur-MAN-PAB,{1}aš-šur-MAN-⸢PAB,1,o 1,,Aššur-šarru-uṣur,PN,P224485.2.4,1,P224485.2.4.1,P224485,4
7,,šur,,Aššur-šarru-uṣur,PN,,P224485.json,{1}aš-šur-MAN-PAB,{1}aš-šur-MAN-⸢PAB,1,o 1,,Aššur-šarru-uṣur,PN,P224485.2.4,1,P224485.2.4.2,P224485,4
8,,MAN,,Aššur-šarru-uṣur,PN,,P224485.json,{1}aš-šur-MAN-PAB,{1}aš-šur-MAN-⸢PAB,1,o 1,,Aššur-šarru-uṣur,PN,P224485.2.4,1,P224485.2.4.3,P224485,4
9,,PAB,damaged,Aššur-šarru-uṣur,PN,,P224485.json,{1}aš-šur-MAN-PAB,{1}aš-šur-MAN-⸢PAB,1,o 1,,Aššur-šarru-uṣur,PN,P224485.2.4,1,P224485.2.4.4,P224485,4


<h3>Setting Up the Data for Clustering</h3>
<p>The general goal is to assign a vector to each text that reflects the usage of variant orthography and paleography.</p>
<ol>
    <li>Paleography - Any one set of wedges that we classify as a sign can be impressed on the clay in different ways. For example, a wedge can be missing or one can be added. Also, the tilt of a wedge can variable. These are the features we want to examine in order to see if one text prefers one sign writing or another.</li>
    <li>Orthography - Due to the homophony of the cuneiform writing system, one syllable can be written with many signs. For example, 'li' can be written with the LI-sign but also with the NI-sign, in which case it would be transliterated as li<sub>2</sub></li>
    <li>Orthography (Words) - In addition to syllables being written by variant sign forms, a word (with appropriate inflection) can be written in different ways similar to how "color" is written "colour" in British English. These two variant orthographies indicate the same word but their variation might indicate something about the author.</li>
</ol>

<p>This section therefore contains three subsections. One groups the diagnostic signs with or without modifications per text to. The next discovers the homophonous signs used throughout the corpus and groups different usages per text. The third groups the word forms in the corpus.</p>

First of all, let's create more columns in the data frame to aid us
<ol>
<li>sign_form - Using the OGSL mapping we created earlier, we can assign each reading of a sign to its sign form. This will help us combine different readings under the same sign form to help us mark variations in paleography</li>
<li>mods_str - Since the data contains three columns currently with information on variable paleography, it would help us to consolidate them into one column</li>
<li>combined - This column combines sign_form and mods_str into one string similar to how the <i>b</i> column is a combination of str_part and num_part</li>
<li>str_part and num_part - In order to determine which signs share a syllabic value, it will be useful to separate the transliterated readings into their string components and numerical components. Once we do this, we can group rows with the same str_part and count up the different usages of homophonous signs</li>
</ol>

In [6]:
file_names = df['file'].unique()
df['sign_form'] = df['b'].apply(lambda x: sign_index.get(x.lower(),'?'))
df['mods_str'] = df['a'] + '.' + df['f']  + '.' + df['m']

import re
def get_num_part(s):
    try:
        n = re.findall(r'[₀₁₂₃₄₅₆₇₈₉]+',s)[0]
        n = n.replace('₀','0').replace('₁','1').replace('₂','2').replace('₃','3').replace('₄','4')
        n = n.replace('₅','5').replace('₆','6').replace('₇','7').replace('₈','8').replace('₉','9')
    except:
        n = 1
    return n
def get_str_part(s):
    try:
        n = re.findall(r'[a-zA-ZšŠṣṢṭṬʾ \(\)0-9]+',s)[0]
    except:
        n = s
    return n
        
df['str_part'] = df['b'].apply(lambda x: get_str_part(x))
df['num_part'] = df['b'].apply(lambda x: get_num_part(x))
df['combined'] = df['sign_form'] + ':' + df['mods_str']
df

,a,b,break,cf,epos,f,file,form,frag,gw,...,ref,sense,sign_loc_id,text_id,word_sign_tot,sign_form,mods_str,str_part,num_part,combined
0,,a,damaged,awātu,N,,P224485.json,a-bat,⸢a⸣-bat,word,...,P224485.2.1,word,P224485.2.1.0,P224485,2,A,..,a,1,A:..
1,,bat,,awātu,N,,P224485.json,a-bat,⸢a⸣-bat,word,...,P224485.2.1,word,P224485.2.1.1,P224485,2,BAD,..,bat,1,BAD:..
2,,LUGAL,,šarru,N,,P224485.json,LUGAL,LUGAL,king,...,P224485.2.2,king,P224485.2.2.0,P224485,1,LUGAL,..,LUGAL,1,LUGAL:..
3,,a,,ana,PRP,,P224485.json,a-na,a-na\t,to,...,P224485.2.3,to,P224485.2.3.0,P224485,2,A,..,a,1,A:..
4,,na,,ana,PRP,t,P224485.json,a-na,a-na\t,to,...,P224485.2.3,to,P224485.2.3.1,P224485,2,NA,.t.,na,1,NA:.t.
5,,1(diš),,Aššur-šarru-uṣur,PN,,P224485.json,{1}aš-šur-MAN-PAB,{1}aš-šur-MAN-⸢PAB,1,...,P224485.2.4,1,P224485.2.4.0,P224485,4,DIŠ,..,1(diš),1,DIŠ:..
6,,aš,,Aššur-šarru-uṣur,PN,,P224485.json,{1}aš-šur-MAN-PAB,{1}aš-šur-MAN-⸢PAB,1,...,P224485.2.4,1,P224485.2.4.1,P224485,4,AŠ,..,aš,1,AŠ:..
7,,šur,,Aššur-šarru-uṣur,PN,,P224485.json,{1}aš-šur-MAN-PAB,{1}aš-šur-MAN-⸢PAB,1,...,P224485.2.4,1,P224485.2.4.2,P224485,4,SUR,..,šur,1,SUR:..
8,,MAN,,Aššur-šarru-uṣur,PN,,P224485.json,{1}aš-šur-MAN-PAB,{1}aš-šur-MAN-⸢PAB,1,...,P224485.2.4,1,P224485.2.4.3,P224485,4,|U.U|,..,MAN,1,|U.U|:..
9,,PAB,damaged,Aššur-šarru-uṣur,PN,,P224485.json,{1}aš-šur-MAN-PAB,{1}aš-šur-MAN-⸢PAB,1,...,P224485.2.4,1,P224485.2.4.4,P224485,4,PAP,..,PAB,1,PAP:..


<h3>Paleography Setup</h3>
<p>The Data Frame we have contains the entire collection of signs in the corpus. However, not every sign has variants in paleography (at least according to Parpola's data input). We only want to look at the signs which have these variants, which we will term diagnostic. In the data, they are the signs that include any type of modification</p>

In [7]:
df2 = df[~(df['mods_str'] == '..')]
list_mod_signs = sorted(list(df2['sign_form'].unique()))

df_paleo = df[df['sign_form'].isin(list_mod_signs)]

#remove damaged signs too
df_paleo = df_paleo[df_paleo['break'] != 'damaged']
df_paleo

,a,b,break,cf,epos,f,file,form,frag,gw,...,ref,sense,sign_loc_id,text_id,word_sign_tot,sign_form,mods_str,str_part,num_part,combined
1,,bat,,awātu,N,,P224485.json,a-bat,⸢a⸣-bat,word,...,P224485.2.1,word,P224485.2.1.1,P224485,2,BAD,..,bat,1,BAD:..
2,,LUGAL,,šarru,N,,P224485.json,LUGAL,LUGAL,king,...,P224485.2.2,king,P224485.2.2.0,P224485,1,LUGAL,..,LUGAL,1,LUGAL:..
3,,a,,ana,PRP,,P224485.json,a-na,a-na\t,to,...,P224485.2.3,to,P224485.2.3.0,P224485,2,A,..,a,1,A:..
4,,na,,ana,PRP,t,P224485.json,a-na,a-na\t,to,...,P224485.2.3,to,P224485.2.3.1,P224485,2,NA,.t.,na,1,NA:.t.
7,,šur,,Aššur-šarru-uṣur,PN,,P224485.json,{1}aš-šur-MAN-PAB,{1}aš-šur-MAN-⸢PAB,1,...,P224485.2.4,1,P224485.2.4.2,P224485,4,SUR,..,šur,1,SUR:..
11,,mu,,šulmu,N,,P224485.json,šul-mu,šul⸣-mu,completeness,...,P224485.2.5,health,P224485.2.5.1,P224485,2,MU,..,mu,1,MU:..
12,,ia,,yâšim,IP,,P224485.json,ia-a-ši,ia-⸢a⸣-ši,to me,...,P224485.2.6,me,P224485.2.6.0,P224485,3,|I.A|,..,ia,1,|I.A|:..
14,,ši,,yâšim,IP,,P224485.json,ia-a-ši,ia-⸢a⸣-ši,to me,...,P224485.2.6,me,P224485.2.6.2,P224485,3,IGI,..,ši,1,IGI:..
16,,mu,,šulmu,N,,P224485.json,šul-mu,⸢šul⸣-mu,completeness,...,P224485.3.1,health,P224485.3.1.1,P224485,2,MU,..,mu,1,MU:..
17,,a,,ana,PRP,,P224485.json,a-na,a-na\t,to,...,P224485.3.2,to,P224485.3.2.0,P224485,2,A,..,a,1,A:..


Create the text matrix with raw counts

In [8]:
df_paleo_str = pd.DataFrame(df_paleo.groupby(['text_id']).apply(lambda x: ' '.join(x['combined'])))
df_paleo_str.columns = ['paleo_str']
df_paleo_str

cv = CountVectorizer(token_pattern='[^ ]+',lowercase=False)
ft = cv.fit_transform(list(df_paleo_str['paleo_str']))
tm_paleo = pd.DataFrame(ft.toarray(),columns=cv.get_feature_names(),index=df_paleo_str.index)
tm_paleo

,A:..,A:.d.,AB@g:..,AB@g:.m.,AB@g:.p.,AB₂:..,AB₂:.d.,AK:..,AK:.d.,AK:.dt.,...,ŠIM:..,ŠIM:.d.,ŠIM:.p.,ŠIM:.t.,ŠU:..,ŠU:.d.,ŠU:.m.,ŠU₂:..,ŠU₂:.d.,ŠU₂:.t.
text_id,,,,,,,,,,,,,,,,,,,,,
P224485,104,0,1,0,0,1,0,1,0,0,...,3,0,0,2,4,0,0,28,0,0
P237089,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
P238649,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
P313416,2,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
P313417,39,0,0,0,0,0,0,0,0,0,...,0,0,0,0,8,0,0,0,0,0
P313419,16,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,2,0,0
P313420,13,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2,0,0
P313421,6,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0
P313422,10,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,3,0,0


<h3>Orthography Syllable Setup</h3>
<p>We now limit the original data frame in different way based on orthography. First we need to figure out which syllabic readings have multiple signs that can render them. We then eliminate capital letter entries because indices on logograms indicate different words and are not relevant here. Last, we limit the data frame to only these signs.

In [9]:
df2 = pd.DataFrame(df.groupby(['str_part'])['num_part'].agg('nunique'))
list_ortho_syls = list(df2[df2[('num_part')] > 1].index)

list_ortho_syls = [h for h in list_ortho_syls if len(re.findall(r'[A-Z]',h)) == 0]
list_ortho_syls

df_ortho_signs = df[df['str_part'].isin(list_ortho_syls)]
df_ortho_signs

,a,b,break,cf,epos,f,file,form,frag,gw,...,ref,sense,sign_loc_id,text_id,word_sign_tot,sign_form,mods_str,str_part,num_part,combined
0,,a,damaged,awātu,N,,P224485.json,a-bat,⸢a⸣-bat,word,...,P224485.2.1,word,P224485.2.1.0,P224485,2,A,..,a,1,A:..
3,,a,,ana,PRP,,P224485.json,a-na,a-na\t,to,...,P224485.2.3,to,P224485.2.3.0,P224485,2,A,..,a,1,A:..
6,,aš,,Aššur-šarru-uṣur,PN,,P224485.json,{1}aš-šur-MAN-PAB,{1}aš-šur-MAN-⸢PAB,1,...,P224485.2.4,1,P224485.2.4.1,P224485,4,AŠ,..,aš,1,AŠ:..
12,,ia,,yâšim,IP,,P224485.json,ia-a-ši,ia-⸢a⸣-ši,to me,...,P224485.2.6,me,P224485.2.6.0,P224485,3,|I.A|,..,ia,1,|I.A|:..
13,,a,damaged,yâšim,IP,,P224485.json,ia-a-ši,ia-⸢a⸣-ši,to me,...,P224485.2.6,me,P224485.2.6.1,P224485,3,A,..,a,1,A:..
17,,a,,ana,PRP,,P224485.json,a-na,a-na\t,to,...,P224485.3.2,to,P224485.3.2.0,P224485,2,A,..,a,1,A:..
20,,aš,,Mat-Aššur,GN,,P224485.json,KUR-aš-šur{KI},KUR-aš-šur{ki},Assyria,...,P224485.3.3,Assyria,P224485.3.3.1,P224485,4,AŠ,..,aš,1,AŠ:..
24,,ka,missing,libbu,N,,P224485.json,ŠA₃-ka,⸢ŠA₃⸣-[ka],interior,...,P224485.3.4,mood,P224485.3.4.1,P224485,2,KA,..,ka,1,KA:..
28,,ka,,ṭābu,AJ,,P224485.json,DUG₃.GA-ka,DUG₃.GA-ka,good,...,P224485.3.6,good,P224485.3.6.2,P224485,2,KA,..,ka,1,KA:..
29,,ša,damaged,ša,REL,,P224485.json,ša,⸢ša⸣,that,...,P224485.4.1,what,P224485.4.1.0,P224485,1,ŠA,..,ša,1,ŠA:..


Create the text matrix with raw counts

In [10]:
df_ortho_str = pd.DataFrame(df_ortho_signs.groupby(['text_id']).apply(lambda x: ' '.join(x['b'])))
df_ortho_str.columns = ['ortho_str']
df_ortho_str

cv = CountVectorizer(token_pattern='[^ ]+',lowercase=False)
ft = cv.fit_transform(list(df_ortho_str['ortho_str']))
tm_ortho_sign = pd.DataFrame(ft.toarray(),columns=cv.get_feature_names(),index=df_ortho_str.index)
tm_ortho_sign

,a,ana,ana₃,ar,ar₂,aš,aš₂,a₂,be,be₂,...,ša₂,šu,šum,šum₂,šu₂,ṭe,ṭe₂,ṭe₃,ṭi,ṭi₂
text_id,,,,,,,,,,,,,,,,,,,,,
P224485,103,2,0,6,0,5,8,0,7,2,...,5,4,1,0,33,0,0,1,0,0
P237089,5,0,0,0,0,0,0,0,2,0,...,0,1,0,0,0,0,0,0,0,0
P238649,5,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,2,0,0
P313416,3,0,0,0,0,0,0,0,3,0,...,0,0,0,0,0,0,0,0,0,0
P313417,36,0,0,0,0,1,2,0,0,0,...,0,8,0,0,0,1,0,1,1,0
P313419,16,0,0,0,0,0,0,0,0,0,...,1,0,0,0,2,0,0,1,0,1
P313420,14,1,0,1,0,1,0,0,6,0,...,0,0,0,0,3,0,0,1,0,0
P313421,7,0,0,0,0,1,0,0,1,0,...,0,0,0,0,1,0,0,0,0,0
P313422,11,1,0,2,0,1,1,0,1,1,...,0,0,0,0,4,0,0,0,0,0


<h3>Orthography Word Setup</h3>
<p>The setup for this section is slightly different. Instead of counting up every sign or reading, we count up every word, its normalization and its form. To help us out we will create two new columns <i>lemma_norm</i> and <i>lemma_norm_form</i>. The first combines the lemma and the normalized form. The latter attaches the transliteration to the former. We only want to include lemma/norm combinations that have multiple forms associated with them.

In [11]:
df_words = pd.DataFrame(all_words)
df_words = df_words[(df_words['cf'] != 'no-cf')]

df_words['lemma'] = df_words['cf'] + '[' + df_words['gw'] + ']' + df_words['pos']
df_words['lemma_norm'] = df_words['lemma'] + ':' + df_words['norm']
df_words['lemma_norm_form'] = df_words['lemma_norm'] + ':' + df_words['form']

df_norm_uniq = pd.DataFrame(df_words.groupby('lemma_norm')['form'].nunique())
list_ortho_words = list(df_norm_uniq[df_norm_uniq[('form')] > 1].index)

df_form_50 = pd.DataFrame(df_words.groupby('lemma_norm_form')['form'].agg('count'))
list_form_50 = list(df_form_50[df_form_50[('form')] > 50].index)

df_ortho_words = df_words[(df_words['lemma_norm'].isin(list_ortho_words)) & df_words['lemma_norm_form'].isin(list_form_50)]
df_ortho_words

,cf,epos,file,form,frag,gw,line_label,norm,pos,ref,sense,text_id,word_sign_tot,lemma,lemma_norm,lemma_norm_form
1,šarru,N,P224485.json,LUGAL,LUGAL,king,o 1,šarri,N,P224485.2.2,king,P224485,1,šarru[king]N,šarru[king]N:šarri,šarru[king]N:šarri:LUGAL
2,ana,PRP,P224485.json,a-na,a-na\t,to,o 1,ana,PRP,P224485.2.3,to,P224485,2,ana[to]PRP,ana[to]PRP:ana,ana[to]PRP:ana:a-na
4,šulmu,N,P224485.json,šul-mu,šul⸣-mu,completeness,o 1,šulmu,N,P224485.2.5,health,P224485,2,šulmu[completeness]N,šulmu[completeness]N:šulmu,šulmu[completeness]N:šulmu:šul-mu
6,šulmu,N,P224485.json,šul-mu,⸢šul⸣-mu,completeness,o 2,šulmu,N,P224485.3.1,health,P224485,2,šulmu[completeness]N,šulmu[completeness]N:šulmu,šulmu[completeness]N:šulmu:šul-mu
7,ana,PRP,P224485.json,a-na,a-na\t,to,o 2,ana,PRP,P224485.3.2,to,P224485,2,ana[to]PRP,ana[to]PRP:ana,ana[to]PRP:ana:a-na
10,lū,MOD,P224485.json,lu,⸢lu⸣,may,o 2,lū,MOD,P224485.3.5,may,P224485,1,lū[may]MOD,lū[may]MOD:lū,lū[may]MOD:lū:lu
12,ša,REL,P224485.json,ša,⸢ša⸣,that,o 3,ša,REL,P224485.4.1,what,P224485,1,ša[that]REL,ša[that]REL:ša,ša[that]REL:ša:ša
14,mā,PRP,P224485.json,ma-a,ma-a,saying,o 3,mā,PRP,P224485.4.3,saying,P224485,2,mā[saying]PRP,mā[saying]PRP:mā,mā[saying]PRP:mā:ma-a
16,ša,DET,P224485.json,ša,[ša],of,o 3,ša,DET,P224485.4.5,of,P224485,1,ša[of]DET,ša[of]DET:ša,ša[of]DET:ša:ša
19,ina,PRP,P224485.json,ina,ina,in,o 4,ina,PRP,P224485.5.2,in,P224485,1,ina[in]PRP,ina[in]PRP:ina,ina[in]PRP:ina:ina


Create the text matrix with raw counts

In [12]:
df_ortho_wordstr = pd.DataFrame(df_ortho_words.groupby(['text_id']).apply(lambda x: ' '.join(x['lemma_norm_form'])))
df_ortho_wordstr.columns = ['ortho_wordstr']
df_ortho_str

cv = CountVectorizer(token_pattern='[^ ]+',lowercase=False)
ft = cv.fit_transform(list(df_ortho_wordstr['ortho_wordstr']))
tm_ortho_word = pd.DataFrame(ft.toarray(),columns=cv.get_feature_names(),index=df_ortho_wordstr.index)
tm_ortho_word.to_csv('output/tm_ortho_word.csv',encoding='utf-8',sep='\t')
tm_ortho_word

,Urarṭaya[Urarṭian]EN:Urarṭaya:{KUR}URI-a.a,adanniš[very,adi[until]PRP:adi:a-di,akī[as]PRP:akī:a-ki,alāku[go]V:ittalka:it-tal-ka,ammar[as,ana[to]PRP:ana:a-na,annûri[now]AV:annurig:an-nu-rig,anāku[I]IP:anāku:a-na-ku,ardu[slave]N:urdaka:ARAD-ka,...,šumma[if]MOD:šumma:šum₂-ma,šumma[if]MOD:šummu:šum₂-mu,šunu[they]IP:šunu:šu-nu,šū[he]IP:šû:šu-u,šū[he]IP:šû:šu-u₂,ūma[today]AV:ūmâ:u₂-ma-a,ṣābu[people]N:ṣābāni:ERIM-MEŠ,ṣābu[people]N:ṣābāni:{LU₂}ERIM-MEŠ,ṭābu[good]AJ:ṭāb:DUG₃.GA,ṭēmu[(fore)thought]N:ṭēmu:ṭe₃-e-mu
text_id,,,,,,,,,,,,,,,,,,,,,
P224485,0,2,0,0,2,1,11,4,0,0,...,0,0,0,0,1,3,3,0,0,0
P237089,0,0,0,0,0,0,2,0,0,1,...,0,0,0,0,0,0,0,0,1,0
P238649,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
P313416,0,0,0,0,0,0,3,0,0,1,...,0,0,0,0,0,0,0,0,0,0
P313417,0,0,0,0,0,1,7,0,0,1,...,0,0,0,0,0,2,1,0,0,1
P313419,0,0,0,0,0,0,5,1,0,1,...,0,0,0,0,0,0,0,0,0,1
P313420,0,0,1,0,0,0,3,0,0,1,...,0,0,0,0,0,0,1,0,0,1
P313421,0,0,0,0,0,0,2,0,0,1,...,0,0,0,0,0,0,0,0,0,0
P313422,0,0,1,0,0,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0


Put them all together

In [13]:
tm_counts = pd.concat([tm_paleo,tm_ortho_sign,tm_ortho_word],axis=1)

<h3>Selecting the Features</h3>

<p>Three criteria are employed for selecting the initial list of features for clustering</p>
<ol>
    <li>Variations that have the highest frequency throughout the corpus</li>
    <li>Variations that are used in a mixed distribution rather than a complementary distribution. This criterion is only for the paleography and syllable variation, whose context is examined through the words they appear in. The form variations themselves thus cannot be examined in this context.</li>
    <li>Variations that look "good" based on an examination of the feature count matrices and their distrubtion within certain groups to be defined later</li>
</ol>
    Obviously, we cannot look at (3) right now but we can look at each of the first two for the three feature types.

<p>1.1 Paleography Frequencies</p>

In [14]:
df_paleo_count = pd.DataFrame(df_paleo.groupby(['sign_form','combined'])['cf'].agg('count'))
df_paleo_count = df_paleo_count[df_paleo_count[('cf')] > 100]

df_paleo_uniq = df_paleo_count.groupby([('sign_form')]).nunique()

df_paleo_count = df_paleo_count.loc[list(df_paleo_uniq[df_paleo_uniq[('cf')] > 1].index)]
df_paleo_count

cf
sign_form  combined            
BA         BA:..            401
           BA:.t.           120
BU         BU:..            960
           BU:.p.           109
DA         DA:..            434
           DA:.d.           201
DI         DI:..           1024
           DI:.d.           156
LI         LI:..            465
           LI:.d.           147
LU₂        LU₂:..           218
           LU₂:v..         1147
           LU₂:v.y.         500
NA         NA:..           2500
           NA:.t.           442
NI         NI:..           4666
           NI:.d.           281
RU         RU:..            444
           RU:.d.           108
SU         SU:..            245
           SU:.t.           116
TA         TA:..            960
           TA:.m.v          309
TI         TI:..            278
           TI:.t.           450
U₂         U₂:..           1302
           U₂:.m.           325
|ME.U.U.U| |ME.U.U.U|:..   1375
           |ME.U.U.U|:.m.   565
ŠA         ŠA:..           1082
           ŠA:.d.           263
           ŠA:.dm.          291
           ŠA:.m.           157
           ŠA:.y.           110
           ŠA:.yd.          124

<p>1.2 Syllable Frequencies</p>

In [15]:
df_ortho_sign_count = pd.DataFrame(df_ortho_signs.groupby(['str_part','b'])['cf'].agg('count'))
df_ortho_sign_count = df_ortho_sign_count[df_ortho_sign_count[('cf')] > 100]

df_ortho_sign_uniq = df_ortho_sign_count.groupby([('str_part')]).nunique()

df_ortho_sign_count = df_ortho_sign_count.loc[list(df_ortho_sign_uniq[df_ortho_sign_uniq[('cf')] > 1].index)]
df_ortho_sign_count

cf
str_part b        
aš       aš    392
         aš₂   427
ia       ia   1724
         ia₂   424
li       li    701
         li₂  1191
tu       tu    440
         tu₂   533
u        u    1503
         u₂   1843
         u₃    256
ša       ša   2827
         ša₂   439
šu       šu    747
         šu₂  1725

<p>1.3 Word Form Frequencies</p>

In [16]:
df_ortho_word_count = pd.DataFrame(df_words.groupby(['lemma_norm','lemma_norm_form'])['cf'].agg('count'))
df_ortho_word_count = df_ortho_word_count[df_ortho_word_count[('cf')] > 50]

df_ortho_word_uniq = df_ortho_word_count.groupby([('lemma_norm')]).nunique()

df_ortho_word_count = df_ortho_word_count.loc[list(df_ortho_word_uniq[df_ortho_word_uniq[('cf')] > 1].index)]
df_ortho_word_count

cf
lemma_norm                 lemma_norm_form                         
bēlu[lord]N:bēlī           bēlu[lord]N:bēlī:EN                  182
                           bēlu[lord]N:bēlī:be-li₂              642
bēlu[lord]N:bēlīya         bēlu[lord]N:bēlīya:EN-a               76
                           bēlu[lord]N:bēlīya:EN-ia             770
                           bēlu[lord]N:bēlīya:EN-ia₂            190
                           bēlu[lord]N:bēlīya:be-li₂-ia         309
                           bēlu[lord]N:bēlīya:be-li₂-ia₂        194
ištu[from]PRP:issu         ištu[from]PRP:issu:TA                113
                           ištu[from]PRP:issu:TA@v              346
libbu[interior]N:libbi     libbu[interior]N:libbi:ŠA₃           217
                           libbu[interior]N:libbi:ŠA₃-bi        277
lā[not]MOD:lā              lā[not]MOD:lā:la                     623
                           lā[not]MOD:lā:la-a                   117
lū[may]MOD:lū              lū[may]MOD:lū:lu                     516
                           lū[may]MOD:lū:lu-u                   190
muhhu[skull]N:muhhi        muhhu[skull]N:muhhi:UGU              681
                           muhhu[skull]N:muhhi:UGU-hi            56
mā[saying]PRP:mā           mā[saying]PRP:mā:ma                   61
                           mā[saying]PRP:mā:ma-a               1442
māru[son]N:mār             māru[son]N:mār:DUMU                   61
                           māru[son]N:mār:{LU₂}A-KIN             51
pānu[front]N:pān           pānu[front]N:pān:IGI                 142
                           pānu[front]N:pān:pa-an               146
šulmu[completeness]N:šulmu šulmu[completeness]N:šulmu:DI-mu     597
                           šulmu[completeness]N:šulmu:šul-mu     91
šū[he]IP:šû                šū[he]IP:šû:šu-u                      52
                           šū[he]IP:šû:šu-u₂                     81
ṣābu[people]N:ṣābāni       ṣābu[people]N:ṣābāni:ERIM-MEŠ         67
                           ṣābu[people]N:ṣābāni:{LU₂}ERIM-MEŠ   148

<p>2.1 Paleography Distrubtion within words

Now let's try to apply a quantitative method to figure out the level of mixed distribution which paleographic variants bear within word forms. The steps here are:
<ol>
    <li>Select only the particular sign forms and modifications that appear a sufficient number of times within the same forms</li>
    <li>Count the number of times these sign forms and modiciations occur in each text</li>
    <li>Create a text matrix which contains the appropriate distribution for the modifications within each sign form, adding 1 to each cell to avoid divide by zero issues</li>
</ol>

In [23]:
df_mods_agg = pd.DataFrame(df_paleo.groupby(['sign_form','form','mods_str'])['a'].agg('count')).reset_index()
df_mods_agg.columns = ['sign_form','form','mods_str','count']
#first let's remove where total instances are less than a certain arbitrary value, say 5
df_mods_agg = df_mods_agg[df_mods_agg['count'] >= 10]
#NOW find and only keep the rows where sign_form and form are duplicates
df_mods_agg['is_dup'] = df_mods_agg.duplicated(['sign_form','form'],False)
df_mods_agg = df_mods_agg[df_mods_agg['is_dup'] == True]
df_mods_agg

,sign_form,form,mods_str,count,is_dup
1919,ANŠE,{ANŠE}KUR.RA-MEŠ,..,32,True
1920,ANŠE,{ANŠE}KUR.RA-MEŠ,.d.,13,True
1988,ARAD,ARAD-ka,..,426,True
1990,ARAD,ARAD-ka,.p.,17,True
2278,BA,i-ba-aš₂-ši,..,28,True
2279,BA,i-ba-aš₂-ši,.t.,11,True
3933,DA,u₂-da,..,39,True
3934,DA,u₂-da,.d.,21,True
4088,DI,DI-mu,..,497,True
4089,DI,DI-mu,.d.,53,True


In [24]:
df_select_signmods = df_mods_agg[['sign_form','mods_str']].drop_duplicates()
df_select_signmods['combined'] = df_select_signmods['sign_form'] + ':' + df_select_signmods['mods_str']
df_select_signmods

,sign_form,mods_str,combined
1919,ANŠE,..,ANŠE:..
1920,ANŠE,.d.,ANŠE:.d.
1988,ARAD,..,ARAD:..
1990,ARAD,.p.,ARAD:.p.
2278,BA,..,BA:..
2279,BA,.t.,BA:.t.
3933,DA,..,DA:..
3934,DA,.d.,DA:.d.
4088,DI,..,DI:..
4089,DI,.d.,DI:.d.


<p>2.2 Same for Syllable Orthography</p>

In [27]:
df_ortho_signs['form_str_part'] = df_ortho_signs['form'].apply(lambda x: re.sub(r'[₁₂₃₄₅₆₇₈₉₀]','',x))
df_ortho_signs

C:\Users\jason\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,a,b,break,cf,epos,f,file,form,frag,gw,...,sense,sign_loc_id,text_id,word_sign_tot,sign_form,mods_str,str_part,num_part,combined,form_str_part
0,,a,damaged,awātu,N,,P224485.json,a-bat,⸢a⸣-bat,word,...,word,P224485.2.1.0,P224485,2,A,..,a,1,A:..,a-bat
3,,a,,ana,PRP,,P224485.json,a-na,a-na\t,to,...,to,P224485.2.3.0,P224485,2,A,..,a,1,A:..,a-na
6,,aš,,Aššur-šarru-uṣur,PN,,P224485.json,{1}aš-šur-MAN-PAB,{1}aš-šur-MAN-⸢PAB,1,...,1,P224485.2.4.1,P224485,4,AŠ,..,aš,1,AŠ:..,{1}aš-šur-MAN-PAB
12,,ia,,yâšim,IP,,P224485.json,ia-a-ši,ia-⸢a⸣-ši,to me,...,me,P224485.2.6.0,P224485,3,|I.A|,..,ia,1,|I.A|:..,ia-a-ši
13,,a,damaged,yâšim,IP,,P224485.json,ia-a-ši,ia-⸢a⸣-ši,to me,...,me,P224485.2.6.1,P224485,3,A,..,a,1,A:..,ia-a-ši
17,,a,,ana,PRP,,P224485.json,a-na,a-na\t,to,...,to,P224485.3.2.0,P224485,2,A,..,a,1,A:..,a-na
20,,aš,,Mat-Aššur,GN,,P224485.json,KUR-aš-šur{KI},KUR-aš-šur{ki},Assyria,...,Assyria,P224485.3.3.1,P224485,4,AŠ,..,aš,1,AŠ:..,KUR-aš-šur{KI}
24,,ka,missing,libbu,N,,P224485.json,ŠA₃-ka,⸢ŠA₃⸣-[ka],interior,...,mood,P224485.3.4.1,P224485,2,KA,..,ka,1,KA:..,ŠA-ka
28,,ka,,ṭābu,AJ,,P224485.json,DUG₃.GA-ka,DUG₃.GA-ka,good,...,good,P224485.3.6.2,P224485,2,KA,..,ka,1,KA:..,DUG.GA-ka
29,,ša,damaged,ša,REL,,P224485.json,ša,⸢ša⸣,that,...,what,P224485.4.1.0,P224485,1,ŠA,..,ša,1,ŠA:..,ša


In [35]:
df_syls_agg = pd.DataFrame(df_ortho_signs.groupby(['str_part','form_str_part','b'])['a'].agg('count')).reset_index()
df_syls_agg.columns = ['str_part','form_str_part','b','count']
#first let's remove where total instances are less than a certain arbitrary value, say 5
df_syls_agg = df_syls_agg[df_syls_agg['count'] >= 10]
#NOW find and only keep the rows where sign_form and form are duplicates
df_syls_agg['is_dup'] = df_syls_agg.duplicated(['str_part','form_str_part'],False)
df_syls_agg = df_syls_agg[df_syls_agg['is_dup'] == True]
df_syls_agg

,str_part,form_str_part,b,count,is_dup
2369,dul,dul-lu,dul,70,True
2370,dul,dul-lu,dul₆,31,True
3264,ia,EN-ia,ia,770,True
3265,ia,EN-ia,ia₂,190,True
3294,ia,be-li-ia,ia,310,True
3295,ia,be-li-ia,ia₂,196,True
4591,li,be-li,li,19,True
4592,li,be-li,li₂,645,True
5198,mi,mi-i-nu,mi,61,True
5199,mi,mi-i-nu,mi₃,14,True


In [29]:
df_select_bs = df_syls_agg[['str_part','b']].drop_duplicates()
#Don't need to create combined column here because b is sufficient
#df_select_signmods['combined'] = df_select_signmods['sign_form'] + ':' + df_select_signmods['mods_str']
df_select_bs

,str_part,b
2369,dul,dul
2370,dul,dul₆
2418,gur,gur
2419,gur,gur₂
3248,ia,ia
3249,ia,ia₂
4591,li,li
4592,li,li₂
5198,mi,mi
5199,mi,mi₃


In [30]:
select_syls = list(df_syls_agg['str_part'].unique())
select_syls

['dul', 'gur', 'ia', 'li', 'mi', 'tu', 'u', 'ša', 'šu', 'šum', 'ṭi']

In [31]:
df_file_select_bs = df_ortho_signs[df_ortho_signs['str_part'].isin(select_syls)]
df_file_select_bs = pd.DataFrame(df_file_select_bs.groupby(['file','str_part','b'])['a'].agg('count')).reset_index()
#Again combined is just b
#df_file_select_syls['combined'] = df_file_select_signs['sign_form'] + ':' + df_file_select_signs['mods_str']
df_file_select_bs

,file,str_part,b,a
0,P224485.json,dul,dul,2
1,P224485.json,ia,ia,16
2,P224485.json,li,li,6
3,P224485.json,li,li₂,7
4,P224485.json,mi,mi,7
5,P224485.json,mi,mi₃,1
6,P224485.json,tu,tu,10
7,P224485.json,tu,tu₂,6
8,P224485.json,u,u,9
9,P224485.json,u,u₂,19


<p>3. Defining our selection</p>

In [32]:
#Paleography
map_paleo = {#'BU':['BU:..','BU:.p.'],
            'DA':['DA:..','DA:.d.']
             'DI':['DI:..','DI:.d.'],
             'LI':['LI:..','LI:.d.'],
             'LU₂':['LU₂:v..','LU₂:v.y.']
             'NA':['NA:..','NA:.t.'],
             'NI':['NI:..','NI:.d.'],
            'U₂':['U₂:..','U₂:.m.'],
             #'RU':['RU:..','RU:.d.'],
             '|ME.U.U.U|':['|ME.U.U.U|:..','|ME.U.U.U|:.m.'],
             'ŠA': ['ŠA:..','ŠA:.dm.']
            }
list_paleo = []
list_paleo_pairs = []
for v in map_paleo.values():
    list_paleo = list_paleo + v
    list_paleo_pairs.append(v)
list_paleo

#Orthography - Sign
map_ortho_sign = {'ia':['ia','ia₂'],
                  #'li':['li','li₂'],
                  'ša':['ša','ša₂'],
                  'šu':['šu','šu₂'],
                  'u':['u','u₂']
                 }
list_ortho_sign = []
list_ortho_sign_pairs = []
for v in map_ortho_sign.values():
    list_ortho_sign = list_ortho_sign + v
    list_ortho_sign_pairs.append(v)
list_ortho_sign

#Orthography - Word
map_ortho_word = {#'bēlu[lord]N:bēlī':['bēlu[lord]N:bēlī:be-li₂','bēlu[lord]N:bēlī:EN'],
                  'bēlu[lord]N:bēlīya':['bēlu[lord]N:bēlīya:EN-ia','bēlu[lord]N:bēlīya:be-li₂-ia'],
                  'lā[not]MOD:lā':['lā[not]MOD:lā:la','lā[not]MOD:lā:la-a'],
                  'lū[may]MOD:lū':['lū[may]MOD:lū:lu','lū[may]MOD:lū:lu-u'],
                  #'šulmu[completeness]N:šulmu':['šulmu[completeness]N:šulmu:DI-mu','šulmu[completeness]N:šulmu:šul-mu'],
                  #'mā[saying]PRP:mā':['mā[saying]PRP:mā:ma','mā[saying]PRP:mā:ma-a']
                }
list_ortho_word = []
list_ortho_word_pairs = []
for v in map_ortho_word.values():
    list_ortho_word = list_ortho_word + v
    list_ortho_word_pairs.append(v)
list_ortho_word_pairs
list_features_all = list_paleo + list_ortho_sign + list_ortho_word
list_pairs_all = list_paleo_pairs + list_ortho_sign_pairs + list_ortho_word_pairs

Put them all together

In [33]:
map_all = map_paleo.copy()
map_all.update(map_ortho_sign)
map_all.update(map_ortho_word)
map_all

{'NA': ['NA:..', 'NA:.t.'],
 'NI': ['NI:..', 'NI:.d.'],
 'bēlu[lord]N:bēlīya': ['bēlu[lord]N:bēlīya:EN-ia',
  'bēlu[lord]N:bēlīya:be-li₂-ia'],
 'ia': ['ia', 'ia₂'],
 'lā[not]MOD:lā': ['lā[not]MOD:lā:la', 'lā[not]MOD:lā:la-a'],
 'lū[may]MOD:lū': ['lū[may]MOD:lū:lu', 'lū[may]MOD:lū:lu-u'],
 'u': ['u', 'u₂'],
 '|ME.U.U.U|': ['|ME.U.U.U|:..', '|ME.U.U.U|:.m.'],
 'ša': ['ša', 'ša₂'],
 'šu': ['šu', 'šu₂']}

Form the Power Set

In [36]:
def p(s):
    powerset = []
    for i in range(2**len(s)):
        subset = [x for j,x in enumerate(s) if (i >> j) & 1]
        powerset.append(subset)
    return powerset

In [37]:
import itertools
list_power = p(list_pairs_all)
list_power = [list(itertools.chain.from_iterable(x)) for x in list_power if x != []]
len(list_power)

1023

In [38]:
df_form_count = pd.DataFrame(df_words[df_words['lemma_norm_form'].isin(list_ortho_word)].groupby(['lemma_norm','lemma_norm_form'])['cf'].agg('count'))
df_form_count

dict_form_count = {}
for ln in map_ortho_word:
    count1 = float(df_form_count.loc[(ln,map_ortho_word[ln][0])]['cf'])
    count2 = float(df_form_count.loc[(ln,map_ortho_word[ln][1])]['cf'])
    tot12 = count1 + count2
    dict_form_count[map_ortho_word[ln][0]] = count1 / tot12
    dict_form_count[map_ortho_word[ln][1]] = count2 / tot12
dict_form_count

df_paleo_count = pd.DataFrame(df[df['combined'].isin(list_paleo)].groupby(['sign_form','combined'])['cf'].agg('count'))
df_paleo_count

dict_paleo_count = {}
for ln in map_paleo:
    count1 = float(df_paleo_count.loc[(ln,map_paleo[ln][0])]['cf'])
    count2 = float(df_paleo_count.loc[(ln,map_paleo[ln][1])]['cf'])
    tot12 = count1 + count2
    dict_paleo_count[map_paleo[ln][0]] = count1 / tot12
    dict_paleo_count[map_paleo[ln][1]] = count2 / tot12
dict_paleo_count

df_sign_count = pd.DataFrame(df[df['b'].isin(list_ortho_sign)].groupby(['str_part','b'])['cf'].agg('count'))
df_sign_count

dict_sign_count = {}
for ln in map_ortho_sign:
    count1 = float(df_sign_count.loc[(ln,map_ortho_sign[ln][0])]['cf'])
    count2 = float(df_sign_count.loc[(ln,map_ortho_sign[ln][1])]['cf'])
    tot12 = count1 + count2
    dict_sign_count[map_ortho_sign[ln][0]] = count1 / tot12
    dict_sign_count[map_ortho_sign[ln][1]] = count2 / tot12
dict_sign_count

{'ia': 0.8026070763500931,
 'ia₂': 0.1973929236499069,
 'u': 0.4491930663478781,
 'u₂': 0.550806933652122,
 'ša': 0.8655848132271893,
 'ša₂': 0.13441518677281078,
 'šu': 0.30218446601941745,
 'šu₂': 0.6978155339805825}

In [39]:
dict_count_all = dict_paleo_count.copy()
dict_count_all.update(dict_sign_count)
dict_count_all.update(dict_form_count)

Form the Distribution Matrix

In [40]:
def form_dist_matrix(tm_count,mapping):
    d = {}
    vecs = {}
    
    listing = []
    for v in mapping.values():
        listing = listing + v
        
    tm_count = tm_count[listing]
    for i, row in tm_count.iterrows():
        d[i] = {}
        for key in mapping:
            key_sum = np.sum(tm_count.loc[i][mapping[key]])
            for v in mapping[key]:
                if key_sum > 0:
                    d[i][v] = tm_count.loc[i][v] / key_sum
                else:
                    d[i][v] = np.nan

    tm_dist = pd.DataFrame(d).transpose()
    return tm_dist

In [41]:
tm_dist_all = form_dist_matrix(tm_counts,map_all)
tm_dist_all

,NA:..,NA:.t.,NI:..,NI:.d.,bēlu[lord]N:bēlīya:EN-ia,bēlu[lord]N:bēlīya:be-li₂-ia,ia,ia₂,lā[not]MOD:lā:la,lā[not]MOD:lā:la-a,lū[may]MOD:lū:lu,lū[may]MOD:lū:lu-u,u,u₂,|ME.U.U.U|:..,|ME.U.U.U|:.m.,ša,ša₂,šu,šu₂
P224485,0.526316,0.473684,0.977273,0.022727,0.000000,1.000000,1.000000,0.000000,1.000000,0.000000,0.875000,0.125000,0.321429,0.678571,0.315789,0.684211,0.807692,0.192308,0.108108,0.891892
P237089,1.000000,0.000000,1.000000,0.000000,NaN,NaN,0.000000,1.000000,NaN,NaN,0.500000,0.500000,1.000000,0.000000,NaN,NaN,1.000000,0.000000,1.000000,0.000000
P238649,1.000000,0.000000,0.600000,0.400000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.500000,0.500000,NaN,NaN,1.000000,0.000000,NaN,NaN
P313416,1.000000,0.000000,0.500000,0.500000,0.000000,1.000000,0.333333,0.666667,NaN,NaN,0.000000,1.000000,0.333333,0.666667,0.000000,1.000000,NaN,NaN,NaN,NaN
P313417,0.357143,0.642857,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,1.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000
P313419,0.833333,0.166667,0.800000,0.200000,1.000000,0.000000,1.000000,0.000000,0.666667,0.333333,1.000000,0.000000,0.666667,0.333333,NaN,NaN,0.750000,0.250000,0.000000,1.000000
P313420,1.000000,0.000000,1.000000,0.000000,NaN,NaN,0.375000,0.625000,1.000000,0.000000,0.500000,0.500000,1.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,1.000000
P313421,1.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.500000,0.500000,NaN,NaN,0.000000,1.000000,0.333333,0.666667,0.000000,1.000000,1.000000,0.000000,0.000000,1.000000
P313422,0.000000,1.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.428571,0.571429,NaN,NaN,1.000000,0.000000,0.000000,1.000000
P313425,0.769231,0.230769,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.800000,0.200000,1.000000,0.000000,0.473684,0.526316,0.250000,0.750000,0.823529,0.176471,1.000000,0.000000


Fill in empty features with their distribution over the entire corpus

In [42]:
for lnf in dict_count_all:
    #print(dict_form_count[lnf])
    tm_dist_all[lnf] = tm_dist_all[lnf].fillna(dict_count_all[lnf])
tm_dist_all

,NA:..,NA:.t.,NI:..,NI:.d.,bēlu[lord]N:bēlīya:EN-ia,bēlu[lord]N:bēlīya:be-li₂-ia,ia,ia₂,lā[not]MOD:lā:la,lā[not]MOD:lā:la-a,lū[may]MOD:lū:lu,lū[may]MOD:lū:lu-u,u,u₂,|ME.U.U.U|:..,|ME.U.U.U|:.m.,ša,ša₂,šu,šu₂
P224485,0.526316,0.473684,0.977273,0.022727,0.000000,1.000000,1.000000,0.000000,1.000000,0.000000,0.875000,0.125000,0.321429,0.678571,0.315789,0.684211,0.807692,0.192308,0.108108,0.891892
P237089,1.000000,0.000000,1.000000,0.000000,0.713624,0.286376,0.000000,1.000000,0.841892,0.158108,0.500000,0.500000,1.000000,0.000000,0.724797,0.275203,1.000000,0.000000,1.000000,0.000000
P238649,1.000000,0.000000,0.600000,0.400000,0.713624,0.286376,0.802607,0.197393,0.841892,0.158108,0.730878,0.269122,0.500000,0.500000,0.724797,0.275203,1.000000,0.000000,0.302184,0.697816
P313416,1.000000,0.000000,0.500000,0.500000,0.000000,1.000000,0.333333,0.666667,0.841892,0.158108,0.000000,1.000000,0.333333,0.666667,0.000000,1.000000,0.865585,0.134415,0.302184,0.697816
P313417,0.357143,0.642857,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,1.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000
P313419,0.833333,0.166667,0.800000,0.200000,1.000000,0.000000,1.000000,0.000000,0.666667,0.333333,1.000000,0.000000,0.666667,0.333333,0.724797,0.275203,0.750000,0.250000,0.000000,1.000000
P313420,1.000000,0.000000,1.000000,0.000000,0.713624,0.286376,0.375000,0.625000,1.000000,0.000000,0.500000,0.500000,1.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,1.000000
P313421,1.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.500000,0.500000,0.841892,0.158108,0.000000,1.000000,0.333333,0.666667,0.000000,1.000000,1.000000,0.000000,0.000000,1.000000
P313422,0.000000,1.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.428571,0.571429,0.724797,0.275203,1.000000,0.000000,0.000000,1.000000
P313425,0.769231,0.230769,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.800000,0.200000,1.000000,0.000000,0.473684,0.526316,0.250000,0.750000,0.823529,0.176471,1.000000,0.000000


<h3>Clustering</h3>

<p>In this section we iterate over each feature grouping in the power set to see which ones give us the best clustering. The measurements we are using are the Elbow Method and the Silhouette Score. These metrics can tell us how many clusters work best for each feature group, if any at all</p>

In [43]:
def calculate_scores(tm_dist,feature_list,flist_str):
    list_scores = []
    c = 0
    for g in feature_list:
        c += 1
        print(str(c) + '. Working on: ' + str(g))
        diff = 0
        decel = 0
        for k in range(1,30):
            d = {}
            km = KMeans(n_clusters=k,max_iter=1000).fit(tm_dist[g])

            sil_score = np.nan
            if k > 1:
                sil_score = silhouette_score(tm_dist[g],labels=km.labels_)

            if k > 2:
                decel = km.inertia_ - inertia  - diff
            if k > 1:
                diff = km.inertia_ - inertia

            inertia = km.inertia_

            d = {'group': str(g),'group_len': len(g), 'k': k,'sil_score': sil_score,'dist': inertia,'diff':diff,'decel':decel}
            list_scores.append(d)

    df_scores = pd.DataFrame(list_scores)
    df_scores.to_csv('output/scores_' + flist_str + '.csv',encoding='utf-8',sep='\t')
    return df_scores

In [44]:
def export_plots(df_scores,feature_list,flist_str):
    c = 0
    for g in feature_list:
        c += 1
        print(str(c) + '. Plotting: ' + str(g))
        df_temp = df_scores[df_scores['group'] == str(g)]
        K = list(df_temp['k'])

        plt.figure(figsize=(12, 12), dpi=120, facecolor='w', edgecolor='k')

        plt.plot(K,df_temp['dist'])
        plt.xticks(K)
        plt.xlabel('k')
        plt.ylabel('Distortion')
        plt.title('Elbow Method: ' + str(g))
        plt.savefig('output/plots/elbow_' + flist_str + '_reg_' + str(c) + '.png')

        plt.clf()

In [45]:
#df_scores = calculate_scores(tm_dist_all,list_ortho_word_power,'ortho_word')
#export_plots(df_scores,list_ortho_word_power,'ortho_word')
#df_scores = calculate_scores(tm_dist_all,list_ortho_sign_power,'ortho_sign')
#export_plots(df_scores,list_ortho_sign_power,'ortho_sign')
#df_scores = calculate_scores(tm_dist_all,list_paleo_power,'paleo')
#export_plots(df_scores,list_paleo_power,'paleo')

<h3>Evaluation</h3>
<p>For this section we will calculate purity scores for the clusters according to three types of classifications: Sender location, dossier, and SAA chapter</p>

In [46]:
cat_file = codecs.open('sargonletters/catalogue.json','r','utf-8')
cat_json = json.load(cat_file)

class_l = []
class_index = []
for pnum in cat_json['members']:
    id_text        = cat_json['members'][pnum].get('id_text','')
    designation    = cat_json['members'][pnum].get('designation','')
    ancient_author = cat_json['members'][pnum].get('ancient_author','')
    dossier        = cat_json['members'][pnum].get('dossier','')
    dossier_nocertain = '.'.join(dossier.split('.')[0:3])
    saa_chap       = '.'.join(dossier.split('.')[0:2])
    senderloc     = cat_json['members'][pnum].get('senderloc','')
    class_d = {'designation': designation,'ancient_author':ancient_author,'dossier':dossier,'dossier_nocertain': dossier_nocertain, 'senderloc':senderloc,'saa_chap':saa_chap}
    class_index.append(id_text)
    class_l.append(class_d)
    
df_class = pd.DataFrame(class_l,index=class_index)
df_class

,ancient_author,designation,dossier,dossier_nocertain,saa_chap,senderloc
P224485,Sargon II,SAA 01 001,SAA01.01.01.a,SAA01.01.01,SAA01.01,Royal Court
P237089,Issar-duri,SAA 15 014,SAA15.01.01.a,SAA15.01.01,SAA15.01,Arrapha
P238649,(unknown),SAA 15 368,SAA15.09.07.c,SAA15.09.07,SAA15.09,uncertain
P313416,Sin-ašared,SAA 01 158,SAA01.07.15.c,SAA01.07.15,SAA01.07,Assyria
P313417,Mannu-ki-Aššur-le’i,SAA 01 233,SAA01.13.01.c,SAA01.13.01,SAA01.13,Guzana
P313419,[...]-ka’’in,SAA 05 040,SAA05.02.03.a,SAA05.02.03,SAA05.02,Tušhan
P313420,Il-yada’,SAA 15 164,SAA15.06.01.a,SAA15.06.01,SAA15.06,Dur-Kurigalzu
P313421,Šarru-emuranni,SAA 15 237,SAA15.07.01.a,SAA15.07.01,SAA15.07,Babylon
P313422,Gabbu-ana-Aššur,SAA 05 114,SAA05.07.01.c,SAA05.07.01,SAA05.07,Kurbail
P313425,Bel-liqbi,SAA 01 179,SAA01.08.03.c,SAA01.08.03,SAA01.08,Zobah


Information for sender location

In [47]:
df_senderloc_certain = df_class[(df_class['senderloc'] != 'uncertain')]

list_senderloc_certain = list(df_senderloc_certain.index)

senderloc_tot = len(df_senderloc_certain['senderloc'].unique())
senderloc_tot

tm_senderloc_certain = tm_dist_all.loc[list(set(list_senderloc_certain) & set(list(tm_dist_all.index)))]
tm_senderloc_certain

,NA:..,NA:.t.,NI:..,NI:.d.,bēlu[lord]N:bēlīya:EN-ia,bēlu[lord]N:bēlīya:be-li₂-ia,ia,ia₂,lā[not]MOD:lā:la,lā[not]MOD:lā:la-a,lū[may]MOD:lū:lu,lū[may]MOD:lū:lu-u,u,u₂,|ME.U.U.U|:..,|ME.U.U.U|:.m.,ša,ša₂,šu,šu₂
P313635,0.000000,1.000000,1.000000,0.000000,0.713624,0.286376,0.802607,0.197393,1.000000,0.000000,0.730878,0.269122,0.449193,0.550807,0.724797,0.275203,0.865585,0.134415,0.500000,0.500000
P313609,1.000000,0.000000,1.000000,0.000000,0.713624,0.286376,1.000000,0.000000,0.841892,0.158108,0.730878,0.269122,0.666667,0.333333,0.724797,0.275203,1.000000,0.000000,0.000000,1.000000
P313425,0.769231,0.230769,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.800000,0.200000,1.000000,0.000000,0.473684,0.526316,0.250000,0.750000,0.823529,0.176471,1.000000,0.000000
P313995,0.852654,0.147346,1.000000,0.000000,0.713624,0.286376,0.802607,0.197393,1.000000,0.000000,0.730878,0.269122,1.000000,0.000000,0.724797,0.275203,1.000000,0.000000,0.302184,0.697816
P334210,1.000000,0.000000,1.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.841892,0.158108,1.000000,0.000000,0.500000,0.500000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000
P334305,0.333333,0.666667,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.841892,0.158108,1.000000,0.000000,0.200000,0.800000,0.000000,1.000000,0.714286,0.285714,1.000000,0.000000
P313483,1.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.800000,0.200000,0.841892,0.158108,0.500000,0.500000,0.222222,0.777778,0.724797,0.275203,1.000000,0.000000,0.333333,0.666667
P334783,0.555556,0.444444,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.500000,0.500000,0.500000,0.500000,0.600000,0.400000,0.666667,0.333333,0.600000,0.400000,0.333333,0.666667
P334139,1.000000,0.000000,1.000000,0.000000,0.333333,0.666667,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.500000,0.500000,1.000000,0.000000,0.500000,0.500000,0.375000,0.625000
P334390,0.413793,0.586207,1.000000,0.000000,0.333333,0.666667,1.000000,0.000000,0.841892,0.158108,1.000000,0.000000,0.400000,0.600000,0.961538,0.038462,0.750000,0.250000,0.302184,0.697816


Information for dossiers

In [48]:
df_dossier_certain_a = df_class[df_class['dossier'].str.contains('.a')]
df_dossier_certain_b = df_class[df_class['dossier'].str.contains('.b')]
df_dossier_certain_c = df_class[df_class['dossier'].str.contains('.c')]

list_dossier_certain_a = list(df_dossier_certain_a.index)
list_dossier_certain_b = list(df_dossier_certain_b.index)
list_dossier_certain_c = list(df_dossier_certain_c.index)

dossier_certain_a_tot = len(df_dossier_certain_a['dossier'].unique())
dossier_certain_b_tot = len(df_dossier_certain_b['dossier'].unique())
dossier_certain_c_tot = len(df_dossier_certain_c['dossier'].unique())

print(str(dossier_certain_a_tot) + ' unique dossiers in ' + str(len(list_dossier_certain_a)) + ' texts')
print(str(dossier_certain_b_tot) + ' unique dossiers in ' + str(len(list_dossier_certain_b)) + ' texts')
print(str(dossier_certain_c_tot) + ' unique dossiers in ' + str(len(list_dossier_certain_c)) + ' texts')
print(str(len(df_class['dossier_nocertain'].unique())))

106 unique dossiers in 276 texts
44 unique dossiers in 69 texts
93 unique dossiers in 609 texts
161


In [49]:
tm_dossier = tm_dist_all.loc[list(set(df_class.index) & set(list(tm_dist_all.index)))]
tm_dossier

,NA:..,NA:.t.,NI:..,NI:.d.,bēlu[lord]N:bēlīya:EN-ia,bēlu[lord]N:bēlīya:be-li₂-ia,ia,ia₂,lā[not]MOD:lā:la,lā[not]MOD:lā:la-a,lū[may]MOD:lū:lu,lū[may]MOD:lū:lu-u,u,u₂,|ME.U.U.U|:..,|ME.U.U.U|:.m.,ša,ša₂,šu,šu₂
P313635,0.000000,1.000000,1.000000,0.000000,0.713624,0.286376,0.802607,0.197393,1.000000,0.000000,0.730878,0.269122,0.449193,0.550807,0.724797,0.275203,0.865585,0.134415,0.500000,0.500000
P313609,1.000000,0.000000,1.000000,0.000000,0.713624,0.286376,1.000000,0.000000,0.841892,0.158108,0.730878,0.269122,0.666667,0.333333,0.724797,0.275203,1.000000,0.000000,0.000000,1.000000
P313425,0.769231,0.230769,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.800000,0.200000,1.000000,0.000000,0.473684,0.526316,0.250000,0.750000,0.823529,0.176471,1.000000,0.000000
P313995,0.852654,0.147346,1.000000,0.000000,0.713624,0.286376,0.802607,0.197393,1.000000,0.000000,0.730878,0.269122,1.000000,0.000000,0.724797,0.275203,1.000000,0.000000,0.302184,0.697816
P334210,1.000000,0.000000,1.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.841892,0.158108,1.000000,0.000000,0.500000,0.500000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000
P334305,0.333333,0.666667,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.841892,0.158108,1.000000,0.000000,0.200000,0.800000,0.000000,1.000000,0.714286,0.285714,1.000000,0.000000
P313483,1.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.800000,0.200000,0.841892,0.158108,0.500000,0.500000,0.222222,0.777778,0.724797,0.275203,1.000000,0.000000,0.333333,0.666667
P334783,0.555556,0.444444,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.500000,0.500000,0.500000,0.500000,0.600000,0.400000,0.666667,0.333333,0.600000,0.400000,0.333333,0.666667
P334139,1.000000,0.000000,1.000000,0.000000,0.333333,0.666667,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.500000,0.500000,1.000000,0.000000,0.500000,0.500000,0.375000,0.625000
P334390,0.413793,0.586207,1.000000,0.000000,0.333333,0.666667,1.000000,0.000000,0.841892,0.158108,1.000000,0.000000,0.400000,0.600000,0.961538,0.038462,0.750000,0.250000,0.302184,0.697816


Information for SAA chapters

In [50]:
saachap_tot = len(df_class['saa_chap'].unique())
saachap_tot

tm_saachap = tm_dist_all.loc[list(set(df_class.index) & set(list(tm_dist_all.index)))]
#tm_saachap
saachap_tot

37

Define the function for evaluation

In [51]:
from random import shuffle
def calculate_purity_score(dict_class,n):
    purity_score = 0
    for s in dict_class:
        cnt = Counter(dict_class[s])
        purity_score += cnt.most_common()[0][1]

    purity_score = purity_score / n
    return purity_score    

def labels_to_dict(labels,pnums):
    d = {}
    for i in range(len(labels)):
        d[pnums[i]] = labels[i]
    return d

In [55]:
def evaluate_clusters(tm_class,feature_list,flist_str,class_type,K):
    list_evals = []
    c = 0
    for g in feature_list:
        c += 1
        print(str(c) + '. Evaluating: ' + str(g))
        km = KMeans(n_clusters=K,max_iter=1000).fit(tm_class[g])
        labels_random = km.labels_.copy()
        shuffle(labels_random)

        #Create Dictionary of cluster => list of pnums
        clustered = {}
        clustered_random = {}
        for i in range(len(km.labels_)):
            if km.labels_[i] in clustered:
                clustered[km.labels_[i]].append(df_class.loc[tm_class.index[i]][class_type])
            else:
                clustered[km.labels_[i]] = [df_class.loc[tm_class.index[i]][class_type]]
                
            if labels_random[i] in clustered_random:
                clustered_random[labels_random[i]].append(df_class.loc[tm_class.index[i]][class_type])
            else:
                clustered_random[labels_random[i]] = [df_class.loc[tm_class.index[i]][class_type]]
        
        #Create Dictionary of Pnum => cluster
        pnum_clusters = labels_to_dict(km.labels_,tm_class.index)
                
        #purity score
        purity_score_class = calculate_purity_score(clustered,len(tm_class.index))
        purity_score_random = calculate_purity_score(clustered_random,len(tm_class.index))
        
        d = {'group':g,
             'purity_class': purity_score_class,
             'purity_random': purity_score_random,
             'group_len': len(g),
             'class_type': class_type,
             'feature_type': flist_str,
             'num_clusters': K}
        list_evals.append(d)

    df_evals = pd.DataFrame(list_evals)
    #Error columns
    df_evals['error_sub'] = df_evals['purity_class'] - df_evals['purity_random']
    df_evals['error_div'] = df_evals['purity_class'] / df_evals['purity_random']
    df_evals.to_csv('output/evaluation_' + class_type + '_' + flist_str + '.csv',encoding='utf-8')
    return df_evals

In [56]:
df_evals_sender = evaluate_clusters(tm_senderloc_certain,list_power,'allpair','senderloc',senderloc_tot)
#df_evals_dossier = evaluate_clusters(tm_dossier,list_power,'allpair','dossier',len(df_class['dossier_nocertain'].unique()))
#df_evals_saachap = evaluate_clusters(tm_saachap,list_power,'allpair','saa_chap',saachap_tot)

1. Evaluating: ['NA:..', 'NA:.t.']
2. Evaluating: ['NI:..', 'NI:.d.']
3. Evaluating: ['NA:..', 'NA:.t.', 'NI:..', 'NI:.d.']
4. Evaluating: ['|ME.U.U.U|:..', '|ME.U.U.U|:.m.']
5. Evaluating: ['NA:..', 'NA:.t.', '|ME.U.U.U|:..', '|ME.U.U.U|:.m.']
6. Evaluating: ['NI:..', 'NI:.d.', '|ME.U.U.U|:..', '|ME.U.U.U|:.m.']
7. Evaluating: ['NA:..', 'NA:.t.', 'NI:..', 'NI:.d.', '|ME.U.U.U|:..', '|ME.U.U.U|:.m.']
8. Evaluating: ['ia', 'ia₂']
9. Evaluating: ['NA:..', 'NA:.t.', 'ia', 'ia₂']
10. Evaluating: ['NI:..', 'NI:.d.', 'ia', 'ia₂']
11. Evaluating: ['NA:..', 'NA:.t.', 'NI:..', 'NI:.d.', 'ia', 'ia₂']
12. Evaluating: ['|ME.U.U.U|:..', '|ME.U.U.U|:.m.', 'ia', 'ia₂']
13. Evaluating: ['NA:..', 'NA:.t.', '|ME.U.U.U|:..', '|ME.U.U.U|:.m.', 'ia', 'ia₂']
14. Evaluating: ['NI:..', 'NI:.d.', '|ME.U.U.U|:..', '|ME.U.U.U|:.m.', 'ia', 'ia₂']
15. Evaluating: ['NA:..', 'NA:.t.', 'NI:..', 'NI:.d.', '|ME.U.U.U|:..', '|ME.U.U.U|:.m.', 'ia', 'ia₂']
16. Evaluating: ['ša', 'ša₂']
17. Evaluating: ['NA:..', 'NA:.t.', 

110. Evaluating: ['NI:..', 'NI:.d.', '|ME.U.U.U|:..', '|ME.U.U.U|:.m.', 'ia', 'ia₂', 'šu', 'šu₂', 'u', 'u₂']
111. Evaluating: ['NA:..', 'NA:.t.', 'NI:..', 'NI:.d.', '|ME.U.U.U|:..', '|ME.U.U.U|:.m.', 'ia', 'ia₂', 'šu', 'šu₂', 'u', 'u₂']
112. Evaluating: ['ša', 'ša₂', 'šu', 'šu₂', 'u', 'u₂']
113. Evaluating: ['NA:..', 'NA:.t.', 'ša', 'ša₂', 'šu', 'šu₂', 'u', 'u₂']
114. Evaluating: ['NI:..', 'NI:.d.', 'ša', 'ša₂', 'šu', 'šu₂', 'u', 'u₂']
115. Evaluating: ['NA:..', 'NA:.t.', 'NI:..', 'NI:.d.', 'ša', 'ša₂', 'šu', 'šu₂', 'u', 'u₂']
116. Evaluating: ['|ME.U.U.U|:..', '|ME.U.U.U|:.m.', 'ša', 'ša₂', 'šu', 'šu₂', 'u', 'u₂']
117. Evaluating: ['NA:..', 'NA:.t.', '|ME.U.U.U|:..', '|ME.U.U.U|:.m.', 'ša', 'ša₂', 'šu', 'šu₂', 'u', 'u₂']
118. Evaluating: ['NI:..', 'NI:.d.', '|ME.U.U.U|:..', '|ME.U.U.U|:.m.', 'ša', 'ša₂', 'šu', 'šu₂', 'u', 'u₂']
119. Evaluating: ['NA:..', 'NA:.t.', 'NI:..', 'NI:.d.', '|ME.U.U.U|:..', '|ME.U.U.U|:.m.', 'ša', 'ša₂', 'šu', 'šu₂', 'u', 'u₂']
120. Evaluating: ['ia', 'ia₂', 

178. Evaluating: ['NI:..', 'NI:.d.', 'ša', 'ša₂', 'šu', 'šu₂', 'bēlu[lord]N:bēlīya:EN-ia', 'bēlu[lord]N:bēlīya:be-li₂-ia']
179. Evaluating: ['NA:..', 'NA:.t.', 'NI:..', 'NI:.d.', 'ša', 'ša₂', 'šu', 'šu₂', 'bēlu[lord]N:bēlīya:EN-ia', 'bēlu[lord]N:bēlīya:be-li₂-ia']
180. Evaluating: ['|ME.U.U.U|:..', '|ME.U.U.U|:.m.', 'ša', 'ša₂', 'šu', 'šu₂', 'bēlu[lord]N:bēlīya:EN-ia', 'bēlu[lord]N:bēlīya:be-li₂-ia']
181. Evaluating: ['NA:..', 'NA:.t.', '|ME.U.U.U|:..', '|ME.U.U.U|:.m.', 'ša', 'ša₂', 'šu', 'šu₂', 'bēlu[lord]N:bēlīya:EN-ia', 'bēlu[lord]N:bēlīya:be-li₂-ia']
182. Evaluating: ['NI:..', 'NI:.d.', '|ME.U.U.U|:..', '|ME.U.U.U|:.m.', 'ša', 'ša₂', 'šu', 'šu₂', 'bēlu[lord]N:bēlīya:EN-ia', 'bēlu[lord]N:bēlīya:be-li₂-ia']
183. Evaluating: ['NA:..', 'NA:.t.', 'NI:..', 'NI:.d.', '|ME.U.U.U|:..', '|ME.U.U.U|:.m.', 'ša', 'ša₂', 'šu', 'šu₂', 'bēlu[lord]N:bēlīya:EN-ia', 'bēlu[lord]N:bēlīya:be-li₂-ia']
184. Evaluating: ['ia', 'ia₂', 'ša', 'ša₂', 'šu', 'šu₂', 'bēlu[lord]N:bēlīya:EN-ia', 'bēlu[lord]N:bēlīy

236. Evaluating: ['|ME.U.U.U|:..', '|ME.U.U.U|:.m.', 'ia', 'ia₂', 'šu', 'šu₂', 'u', 'u₂', 'bēlu[lord]N:bēlīya:EN-ia', 'bēlu[lord]N:bēlīya:be-li₂-ia']
237. Evaluating: ['NA:..', 'NA:.t.', '|ME.U.U.U|:..', '|ME.U.U.U|:.m.', 'ia', 'ia₂', 'šu', 'šu₂', 'u', 'u₂', 'bēlu[lord]N:bēlīya:EN-ia', 'bēlu[lord]N:bēlīya:be-li₂-ia']
238. Evaluating: ['NI:..', 'NI:.d.', '|ME.U.U.U|:..', '|ME.U.U.U|:.m.', 'ia', 'ia₂', 'šu', 'šu₂', 'u', 'u₂', 'bēlu[lord]N:bēlīya:EN-ia', 'bēlu[lord]N:bēlīya:be-li₂-ia']
239. Evaluating: ['NA:..', 'NA:.t.', 'NI:..', 'NI:.d.', '|ME.U.U.U|:..', '|ME.U.U.U|:.m.', 'ia', 'ia₂', 'šu', 'šu₂', 'u', 'u₂', 'bēlu[lord]N:bēlīya:EN-ia', 'bēlu[lord]N:bēlīya:be-li₂-ia']
240. Evaluating: ['ša', 'ša₂', 'šu', 'šu₂', 'u', 'u₂', 'bēlu[lord]N:bēlīya:EN-ia', 'bēlu[lord]N:bēlīya:be-li₂-ia']
241. Evaluating: ['NA:..', 'NA:.t.', 'ša', 'ša₂', 'šu', 'šu₂', 'u', 'u₂', 'bēlu[lord]N:bēlīya:EN-ia', 'bēlu[lord]N:bēlīya:be-li₂-ia']
242. Evaluating: ['NI:..', 'NI:.d.', 'ša', 'ša₂', 'šu', 'šu₂', 'u', 'u₂', '

302. Evaluating: ['NI:..', 'NI:.d.', '|ME.U.U.U|:..', '|ME.U.U.U|:.m.', 'ia', 'ia₂', 'šu', 'šu₂', 'lā[not]MOD:lā:la', 'lā[not]MOD:lā:la-a']
303. Evaluating: ['NA:..', 'NA:.t.', 'NI:..', 'NI:.d.', '|ME.U.U.U|:..', '|ME.U.U.U|:.m.', 'ia', 'ia₂', 'šu', 'šu₂', 'lā[not]MOD:lā:la', 'lā[not]MOD:lā:la-a']
304. Evaluating: ['ša', 'ša₂', 'šu', 'šu₂', 'lā[not]MOD:lā:la', 'lā[not]MOD:lā:la-a']
305. Evaluating: ['NA:..', 'NA:.t.', 'ša', 'ša₂', 'šu', 'šu₂', 'lā[not]MOD:lā:la', 'lā[not]MOD:lā:la-a']
306. Evaluating: ['NI:..', 'NI:.d.', 'ša', 'ša₂', 'šu', 'šu₂', 'lā[not]MOD:lā:la', 'lā[not]MOD:lā:la-a']
307. Evaluating: ['NA:..', 'NA:.t.', 'NI:..', 'NI:.d.', 'ša', 'ša₂', 'šu', 'šu₂', 'lā[not]MOD:lā:la', 'lā[not]MOD:lā:la-a']
308. Evaluating: ['|ME.U.U.U|:..', '|ME.U.U.U|:.m.', 'ša', 'ša₂', 'šu', 'šu₂', 'lā[not]MOD:lā:la', 'lā[not]MOD:lā:la-a']
309. Evaluating: ['NA:..', 'NA:.t.', '|ME.U.U.U|:..', '|ME.U.U.U|:.m.', 'ša', 'ša₂', 'šu', 'šu₂', 'lā[not]MOD:lā:la', 'lā[not]MOD:lā:la-a']
310. Evaluating: ['N

...

836. Evaluating: ['|ME.U.U.U|:..', '|ME.U.U.U|:.m.', 'u', 'u₂', 'lā[not]MOD:lā:la', 'lā[not]MOD:lā:la-a', 'lū[may]MOD:lū:lu', 'lū[may]MOD:lū:lu-u']
837. Evaluating: ['NA:..', 'NA:.t.', '|ME.U.U.U|:..', '|ME.U.U.U|:.m.', 'u', 'u₂', 'lā[not]MOD:lā:la', 'lā[not]MOD:lā:la-a', 'lū[may]MOD:lū:lu', 'lū[may]MOD:lū:lu-u']
838. Evaluating: ['NI:..', 'NI:.d.', '|ME.U.U.U|:..', '|ME.U.U.U|:.m.', 'u', 'u₂', 'lā[not]MOD:lā:la', 'lā[not]MOD:lā:la-a', 'lū[may]MOD:lū:lu', 'lū[may]MOD:lū:lu-u']
839. Evaluating: ['NA:..', 'NA:.t.', 'NI:..', 'NI:.d.', '|ME.U.U.U|:..', '|ME.U.U.U|:.m.', 'u', 'u₂', 'lā[not]MOD:lā:la', 'lā[not]MOD:lā:la-a', 'lū[may]MOD:lū:lu', 'lū[may]MOD:lū:lu-u']
840. Evaluating: ['ia', 'ia₂', 'u', 'u₂', 'lā[not]MOD:lā:la', 'lā[not]MOD:lā:la-a', 'lū[may]MOD:lū:lu', 'lū[may]MOD:lū:lu-u']
841. Evaluating: ['NA:..', 'NA:.t.', 'ia', 'ia₂', 'u', 'u₂', 'lā[not]MOD:lā:la', 'lā[not]MOD:lā:la-a', 'lū[may]MOD:lū:lu', 'lū[may]MOD:lū:lu-u']
842. Evaluating: ['NI:..', 'NI:.d.', 'ia', 'ia₂', 'u', 'u₂', 

886. Evaluating: ['NI:..', 'NI:.d.', '|ME.U.U.U|:..', '|ME.U.U.U|:.m.', 'ša', 'ša₂', 'šu', 'šu₂', 'u', 'u₂', 'lā[not]MOD:lā:la', 'lā[not]MOD:lā:la-a', 'lū[may]MOD:lū:lu', 'lū[may]MOD:lū:lu-u']
887. Evaluating: ['NA:..', 'NA:.t.', 'NI:..', 'NI:.d.', '|ME.U.U.U|:..', '|ME.U.U.U|:.m.', 'ša', 'ša₂', 'šu', 'šu₂', 'u', 'u₂', 'lā[not]MOD:lā:la', 'lā[not]MOD:lā:la-a', 'lū[may]MOD:lū:lu', 'lū[may]MOD:lū:lu-u']
888. Evaluating: ['ia', 'ia₂', 'ša', 'ša₂', 'šu', 'šu₂', 'u', 'u₂', 'lā[not]MOD:lā:la', 'lā[not]MOD:lā:la-a', 'lū[may]MOD:lū:lu', 'lū[may]MOD:lū:lu-u']
889. Evaluating: ['NA:..', 'NA:.t.', 'ia', 'ia₂', 'ša', 'ša₂', 'šu', 'šu₂', 'u', 'u₂', 'lā[not]MOD:lā:la', 'lā[not]MOD:lā:la-a', 'lū[may]MOD:lū:lu', 'lū[may]MOD:lū:lu-u']
890. Evaluating: ['NI:..', 'NI:.d.', 'ia', 'ia₂', 'ša', 'ša₂', 'šu', 'šu₂', 'u', 'u₂', 'lā[not]MOD:lā:la', 'lā[not]MOD:lā:la-a', 'lū[may]MOD:lū:lu', 'lū[may]MOD:lū:lu-u']
891. Evaluating: ['NA:..', 'NA:.t.', 'NI:..', 'NI:.d.', 'ia', 'ia₂', 'ša', 'ša₂', 'šu', 'šu₂', 'u', '

927. Evaluating: ['NA:..', 'NA:.t.', 'NI:..', 'NI:.d.', '|ME.U.U.U|:..', '|ME.U.U.U|:.m.', 'ia', 'ia₂', 'ša', 'ša₂', 'bēlu[lord]N:bēlīya:EN-ia', 'bēlu[lord]N:bēlīya:be-li₂-ia', 'lā[not]MOD:lā:la', 'lā[not]MOD:lā:la-a', 'lū[may]MOD:lū:lu', 'lū[may]MOD:lū:lu-u']
928. Evaluating: ['šu', 'šu₂', 'bēlu[lord]N:bēlīya:EN-ia', 'bēlu[lord]N:bēlīya:be-li₂-ia', 'lā[not]MOD:lā:la', 'lā[not]MOD:lā:la-a', 'lū[may]MOD:lū:lu', 'lū[may]MOD:lū:lu-u']
929. Evaluating: ['NA:..', 'NA:.t.', 'šu', 'šu₂', 'bēlu[lord]N:bēlīya:EN-ia', 'bēlu[lord]N:bēlīya:be-li₂-ia', 'lā[not]MOD:lā:la', 'lā[not]MOD:lā:la-a', 'lū[may]MOD:lū:lu', 'lū[may]MOD:lū:lu-u']
930. Evaluating: ['NI:..', 'NI:.d.', 'šu', 'šu₂', 'bēlu[lord]N:bēlīya:EN-ia', 'bēlu[lord]N:bēlīya:be-li₂-ia', 'lā[not]MOD:lā:la', 'lā[not]MOD:lā:la-a', 'lū[may]MOD:lū:lu', 'lū[may]MOD:lū:lu-u']
931. Evaluating: ['NA:..', 'NA:.t.', 'NI:..', 'NI:.d.', 'šu', 'šu₂', 'bēlu[lord]N:bēlīya:EN-ia', 'bēlu[lord]N:bēlīya:be-li₂-ia', 'lā[not]MOD:lā:la', 'lā[not]MOD:lā:la-a', 'lū[m

964. Evaluating: ['|ME.U.U.U|:..', '|ME.U.U.U|:.m.', 'u', 'u₂', 'bēlu[lord]N:bēlīya:EN-ia', 'bēlu[lord]N:bēlīya:be-li₂-ia', 'lā[not]MOD:lā:la', 'lā[not]MOD:lā:la-a', 'lū[may]MOD:lū:lu', 'lū[may]MOD:lū:lu-u']
965. Evaluating: ['NA:..', 'NA:.t.', '|ME.U.U.U|:..', '|ME.U.U.U|:.m.', 'u', 'u₂', 'bēlu[lord]N:bēlīya:EN-ia', 'bēlu[lord]N:bēlīya:be-li₂-ia', 'lā[not]MOD:lā:la', 'lā[not]MOD:lā:la-a', 'lū[may]MOD:lū:lu', 'lū[may]MOD:lū:lu-u']
966. Evaluating: ['NI:..', 'NI:.d.', '|ME.U.U.U|:..', '|ME.U.U.U|:.m.', 'u', 'u₂', 'bēlu[lord]N:bēlīya:EN-ia', 'bēlu[lord]N:bēlīya:be-li₂-ia', 'lā[not]MOD:lā:la', 'lā[not]MOD:lā:la-a', 'lū[may]MOD:lū:lu', 'lū[may]MOD:lū:lu-u']
967. Evaluating: ['NA:..', 'NA:.t.', 'NI:..', 'NI:.d.', '|ME.U.U.U|:..', '|ME.U.U.U|:.m.', 'u', 'u₂', 'bēlu[lord]N:bēlīya:EN-ia', 'bēlu[lord]N:bēlīya:be-li₂-ia', 'lā[not]MOD:lā:la', 'lā[not]MOD:lā:la-a', 'lū[may]MOD:lū:lu', 'lū[may]MOD:lū:lu-u']
968. Evaluating: ['ia', 'ia₂', 'u', 'u₂', 'bēlu[lord]N:bēlīya:EN-ia', 'bēlu[lord]N:bēlīya:be

1001. Evaluating: ['NA:..', 'NA:.t.', 'ia', 'ia₂', 'šu', 'šu₂', 'u', 'u₂', 'bēlu[lord]N:bēlīya:EN-ia', 'bēlu[lord]N:bēlīya:be-li₂-ia', 'lā[not]MOD:lā:la', 'lā[not]MOD:lā:la-a', 'lū[may]MOD:lū:lu', 'lū[may]MOD:lū:lu-u']
1002. Evaluating: ['NI:..', 'NI:.d.', 'ia', 'ia₂', 'šu', 'šu₂', 'u', 'u₂', 'bēlu[lord]N:bēlīya:EN-ia', 'bēlu[lord]N:bēlīya:be-li₂-ia', 'lā[not]MOD:lā:la', 'lā[not]MOD:lā:la-a', 'lū[may]MOD:lū:lu', 'lū[may]MOD:lū:lu-u']
1003. Evaluating: ['NA:..', 'NA:.t.', 'NI:..', 'NI:.d.', 'ia', 'ia₂', 'šu', 'šu₂', 'u', 'u₂', 'bēlu[lord]N:bēlīya:EN-ia', 'bēlu[lord]N:bēlīya:be-li₂-ia', 'lā[not]MOD:lā:la', 'lā[not]MOD:lā:la-a', 'lū[may]MOD:lū:lu', 'lū[may]MOD:lū:lu-u']
1004. Evaluating: ['|ME.U.U.U|:..', '|ME.U.U.U|:.m.', 'ia', 'ia₂', 'šu', 'šu₂', 'u', 'u₂', 'bēlu[lord]N:bēlīya:EN-ia', 'bēlu[lord]N:bēlīya:be-li₂-ia', 'lā[not]MOD:lā:la', 'lā[not]MOD:lā:la-a', 'lū[may]MOD:lū:lu', 'lū[may]MOD:lū:lu-u']
1005. Evaluating: ['NA:..', 'NA:.t.', '|ME.U.U.U|:..', '|ME.U.U.U|:.m.', 'ia', 'ia₂', 'šu

In [ ]:
df_evals_all = pd.concat([df_evals_sender_ortho_word,
                          df_evals_sender_ortho_sign,
                         df_evals_sender_paleo,
                          df_evals_dossier_ortho_word,
                          df_evals_dossier_ortho_sign,
                          df_evals_dossier_paleo,
                         df_evals_saachap_ortho_word,
                         df_evals_saachap_ortho_sign,
                         df_evals_saachap_paleo])
df_evals_all.to_csv('output/evaluation_all.csv',encoding='utf-8')